In [1]:
import cudf as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
data_for_regression = pd.read_csv('data/all_data_groupped.csv', index_col=0)

In [3]:
data_for_regression['conversion_total'] = data_for_regression[['campaign_type_conversion','event_type_conversion']].sum(axis=1)

In [4]:
def check_if_conversion(df:pd.DataFrame):
    if df['conversion_total'] > 0:
        return 1
    else:
        return 0

In [5]:
data_for_regression['true_if_conversion'] = data_for_regression.apply(check_if_conversion, axis=1)

In [6]:
data_for_regression.drop(['payout', 'user_phone', 'campaign_type_conversion', 'event_type_conversion', 'event_status_approve', 'event_status_deal', 'event_status_payout', 'event_status_reject', 'event_status_sale', 'conversion_total'], axis=1, inplace=True)

In [7]:
data_for_regression.head()

,campaign_cost,campaign_type_click,campaign_type_show,source_medium_android / mobile_app,source_medium_bing / organic,source_medium_email / email,source_medium_finance.rambler.ru / cpm,source_medium_google / organic,source_medium_ios / mobile_app,source_medium_not-set / none,...,source_medium_zen.yandex.ru / cpm,interface_api,interface_app,interface_web,event_type_click,event_type_show,product_type_deposits,product_type_loan,product_type_mortgage,true_if_conversion
0,0.0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
1,0.0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,2,0,0
2,0.0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,1,0,0
3,0.0,0,1,0,0,0,0,0,1,0,...,0,0,1,0,1,1,0,2,0,0
4,0.0,0,2,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,2,0,0


In [8]:
X = data_for_regression.drop('true_if_conversion', axis=1).to_numpy()

In [9]:
y = data_for_regression['true_if_conversion'].to_numpy()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
with open('model.pkl', 'rb') as f:
    model1 = pickle.load(f)

In [12]:
pred1 = model1.predict(X_test)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.2s finished


In [13]:
compare_pred = pd.DataFrame(
    {
        'y_test': y_test,
        'prediction': pred1.astype('int')
    }
)

In [14]:
model1.score(X_test, y_test)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.2s finished


0.9986259316653563

In [15]:
def iftrue(df:pd.DataFrame):
    if df['y_test'] == df['prediction']:
        return True
    else:
        return False

In [16]:
compare_pred['if_true'] = compare_pred.apply(iftrue, axis=1)

In [17]:
compare_pred.sample(20)

,y_test,prediction,if_true
1495683,0,0,True
1614126,0,0,True
917691,0,0,True
707379,0,0,True
1702119,0,0,True
1238773,0,0,True
293245,0,0,True
1416576,0,0,True
79959,0,0,True
437544,0,0,True


In [18]:
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model1, f)